In [1]:
import torch
import torch.nn as nn
from torchvision import datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from torchvision import models
import torch.nn.functional as F
from torch.nn import init
import torchvision as tv
from torch.optim import lr_scheduler


import datetime
import time
from tqdm import tqdm 
import pandas as pd

import numpy as np
from PIL import Image
import os
import datetime
import time
from pathlib import Path
import h5py

from sklearn.metrics import precision_score,f1_score,roc_curve,auc,accuracy_score
from models.gaussian_blur import GaussianBlur
from models.gamma_correction import GammaCorrection,HistogramEqualization
from models.densenet import DenseNet
from models import adamw, compare_auc_delong_xu, cosine_scheduler

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt 
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")

In [2]:
class Config(object):
    def __init__(self):
        self.name = 'Integrated'
        self.dataset_name = 'mimic'
        self.dataroot ='./DATASET/'
        
        self.save_path = './checkpoint/' + self.name
        self.model_path = self.save_path + '/models'
        self.decode_path = self.save_path + '/decoded_results'
        self.val_path = self.save_path + '/val_results'
        self.test_path = self.save_path + '/test_results'
        
        self.num_threads = 8
        self.shuffle_dataset=True
        self.random_seed=24


        self.lr = 0.0005      
        
        self.serial_batches = False
        self.phase='train'
        
        self.batch_size = 6
        self.batch_size = 6
        self.test_batch_size = 1
        self.max_epochs = 500
        self.save_every = 1     #epoch
        self.plot_every = 1     # epoch to save decoded images

        os.makedirs(self.save_path, exist_ok=True)
        os.makedirs(self.model_path, exist_ok=True)
        os.makedirs(self.decode_path, exist_ok=True)
        os.makedirs(self.val_path, exist_ok=True)
        os.makedirs(self.test_path, exist_ok=True)
opt = Config()

In [3]:
phases = ['train', 'val','test']

In [4]:
root_dir = opt.dataroot
raw_data_dir = f'{opt.dataroot}images/'
int_data_dir = f'{root_dir}interim/'
proc_data_dir = f'{root_dir}processed/'
os.makedirs(proc_data_dir, exist_ok=True)
os.makedirs(int_data_dir, exist_ok=True)


# Path to csvfiles on training,validation and tetsing data
csvpath = {phase: f'{root_dir}mimic_csv/{phase}.csv' for phase in phases}

# Load data into dictionary of three Pandas DataFrames
dframe = {phase: pd.read_csv(csvpath[phase]) for phase in phases}

# Calculate sizes
dataset_sizes = {phase: len(dframe[phase]) for phase in phases}
print(dframe['train'].shape, dframe['val'].shape,dframe['test'].shape)

input_size=224
num_classes = 4
num_samples = dframe['train'].shape[0]

# Class names that will be predicted
class_names = dframe['train'].iloc[:, 1:].columns
print(class_names)

# indices we will calculate AUC for, as in the CheXpert paper
competition_tasks = torch.ByteTensor([1, 1, 1, 1]).bool()

df = dframe['train'].iloc[:, 1:].copy()
pos_weight = torch.Tensor([df[cl].sum() / df.shape[0] for cl in class_names])


labels_array = df.iloc[:, 1:].copy()
labels_array = {phase: dframe[phase].iloc[:, 1:].copy() for phase in phases}
for phase in labels_array.keys():
    labels_array[phase] = torch.FloatTensor(labels_array[phase].to_numpy())

# labels_array = torch.FloatTensor(labels_array.to_numpy())  # needed when using cross-entropy loss
print(labels_array['train'].shape)

tforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        #GammaCorrection(0.8),
        #HistogramEqualization(),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])}
hdf50 = Path(f'{int_data_dir}/{phases[0]}_processed.h5')
hdf51 = Path(f'{int_data_dir}/{phases[1]}_processed.h5')
hdf52 = Path(f'{int_data_dir}/{phases[2]}_processed.h5')

(30000, 5) (5000, 5) (1976, 5)
Index(['Consolidation', 'Lung Opacity', 'Pleural Effusion', 'No Findings'], dtype='object')
torch.Size([30000, 4])


In [5]:
def proc_images(img_paths, labels_array,input_size, phases=None, tforms=None):
    """
    Saves compressed, resized images as HDF5 datsets
    Returns
        data.h5, where each dataset is an image or class label
        e.g. X23,y23 = image and corresponding class labels
    """
    if phases is None:
        phases = ['train','val','test']
    for phase in phases:
        print(f'Processing {phase} files...')
        with h5py.File(f'{proc_data_dir}/{phase}_processed.h5', 'w') as hf:
            for i, img_path in enumerate(img_paths[phase]):
                if i % 1000 == 0:
                    print(f"{i} images processed")

                img = Image.open(img_path+'.jpg').convert('RGB')
                if tforms:
                    img = tforms[phase](img)
                hf.create_dataset(
                    name=f"X{i}",
                    data=img,
                    shape=(3, input_size, input_size),
                    maxshape=(3, input_size, input_size),
                    compression="lzf",
                    shuffle="True")
                # Labels
                hf.create_dataset(
                    name=f"y{i}",
                    data=labels_array[phase][i, :],
                    shape=(num_classes,),
                    maxshape=(num_classes,),
                    compression="lzf",
                    shuffle="True",
                    dtype="i1")
    print('Finished!')



In [6]:
hdf50 = Path(f'{proc_data_dir}/{phases[0]}_processed.h5')
hdf51 = Path(f'{proc_data_dir}/{phases[1]}_processed.h5')
hdf52 = Path(f'{proc_data_dir}/{phases[2]}_processed.h5')

In [7]:
if not (hdf50.is_file() or hdf51.is_file() or hdf52.is_file() ):
    img_paths = {phase: raw_data_dir + dframe[phase].iloc[:, 0] for phase in phases}
    proc_images(img_paths, labels_array,input_size,phases=phases, tforms=tforms)
    
hdf5paths = {phase: f'{proc_data_dir}{phase}_processed.h5' for phase in phases}  

In [8]:
class CheXpertDataset(data.Dataset):
    """CheXpert dataset."""

    def __init__(self, phase, num_samples, datapath):
     
        self.phase = phase
        self.num_samples = num_samples
        self.datapath = datapath
        self.data = None

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        if not self.data:    # open in thread
            self.data = h5py.File(f'{self.datapath}', 'r')
#         img = np.float32(self.data[f"X{idx}"][()])
        img = self.data[f"X{idx}"][()] 

        labels = np.float32(self.data[f"y{idx}"][()])

        return img, labels

In [9]:
datasets = {
    phase: CheXpertDataset(phase=phase, num_samples=dataset_sizes[phase],datapath=hdf5paths[phase])
    for phase in phases}

In [10]:
num_workers = 1  # Use with hdf5 files
params = {'train': {'batch_size': opt.batch_size,
                    'shuffle': True,
                    'num_workers': num_workers,
                    'pin_memory': True,
                    'sampler': None,
                    'drop_last':True},
          'val': {'batch_size': opt.batch_size,
                  'shuffle': True,
                  'num_workers': num_workers,
                  'pin_memory': True,
                   'drop_last':True},
           'test': {'batch_size': opt.test_batch_size,
                  'shuffle': False,
                  'num_workers': num_workers,
                  'pin_memory': True,
                   'drop_last':True}
         }

if params['train']['sampler'] is not None:
    params['train']['shuffle'] = False
    
dataloaders = {phase: data.DataLoader(datasets[phase], **params[phase]) for phase in phases}

In [11]:
def down_pooling():
    return nn.MaxPool2d(2)

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.n_z=512      # number of dimensions in latent space.
        
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(inplace=True)
                                )
        self.conv11 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(inplace=True)
                                )
        
        self.conv2=nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(128),
                                 nn.ReLU(inplace=True)
                                )
        self.conv22=nn.Sequential(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(128),
                                 nn.ReLU(inplace=True)
                                )
        self.conv3=nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(256),
                                 nn.LeakyReLU(inplace=True)
                                )
        self.conv33=nn.Sequential(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(256),
                                 nn.ReLU(inplace=True)
                                )
        self.conv4=nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(512),
                                 nn.ReLU(inplace=True)
                                )
        self.conv44=nn.Sequential(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(512),
                                 nn.ReLU(inplace=True)
                                )        
        self.conv5=nn.Sequential(nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(1024),
                                 nn.ReLU(inplace=True)
                                )
        self.conv55=nn.Sequential(nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3,stride=1,padding=1),
                                 nn.BatchNorm2d(1024),
                                 nn.ReLU(inplace=True)
                                )
        self.down_pooling = nn.MaxPool2d(2)
        
        self.mu = nn.Linear(1024*14*14, 256)
        self.logvar = nn.Linear(1024*14*14, 256)

        
    
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample

        
    def forward(self,x):
        
        x=self.conv1(x)
        x=self.conv11(x)
        x = self.down_pooling(x)
        
        x=self.conv2(x)
        x=self.conv22(x)
        x = self.down_pooling(x)
        
        
        x=self.conv3(x)
        x=self.conv33(x)
        x = self.down_pooling(x)

        
        x=self.conv4(x)
        x=self.conv44(x)
        x = self.down_pooling(x)
        
        x=self.conv5(x)
        x=self.conv55(x)
        
        x = x.view(x.size(0),-1)
        mu, logvar = self.mu(x), self.logvar(x)
        x = self.reparameterize(mu, logvar)
        
        return x,mu,logvar




In [12]:
def up_pooling(in_channels, out_channels, kernel_size=2, stride=2):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )
def conv_bn_leru(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
    )

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        
        self.fc = nn.Sequential(nn.Linear(256,1024*14*14),
                                nn.ReLU()
                                )
        self.up_pool6 = up_pooling(1024, 1024)
        self.conv6 = conv_bn_leru(1024, 512)
        self.up_pool7 = up_pooling(512, 512)
        self.conv7 = conv_bn_leru(512, 256)
        self.up_pool8 = up_pooling(256, 256)
        self.conv8 = conv_bn_leru(256, 128)
        self.up_pool9 = up_pooling(128, 128)
        self.conv9 = conv_bn_leru(128, 64)

        self.conv10 = nn.Conv2d(64, 1,1)
        self.relu = nn.ReLU()
    
        
    def forward(self,x):
        x  = self.fc(x)
        x5 = x.view(x.size(0),1024,14,14)
        
        p6 = self.up_pool6(x5)
        x6 = self.conv6(p6)

        p7 = self.up_pool7(x6)
        x7 = self.conv7(p7)

        p8 = self.up_pool8(x7)
        x8 = self.conv8(p8)

        p9 = self.up_pool9(x8)
        x9 = self.conv9(p9)
        
        output = self.conv10(x9)
        output = self.relu(output)
        return output


In [13]:
def free_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = True

def frozen_params(module: nn.Module):
    for p in module.parameters():
        p.requires_grad = False

In [14]:
encoder=Encoder()
decoder=Decoder()

encoder = encoder.to(device)
# decoder = decoder.to(device)
frozen_params(encoder)

In [15]:
def weight_init(m):
    if isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight)
        init.constant_(m.bias, 0)
    if isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight)
        init.constant_(m.bias, 0)

In [16]:
# # define the descriminator
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.n_z = 256
        self.main = nn.Sequential(
            nn.Linear(self.n_z, 256),
            nn.ReLU(True),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Linear(128, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 64),
            nn.ReLU(True),
            nn.Linear(64, 32),
            nn.ReLU(True),
            nn.Linear(32, 32),
            nn.ReLU(True),
            nn.Linear(32, 16),
            nn.ReLU(True),
            nn.Linear(16, 4),
#             nn.Sigmoid()
        )

    def forward(self, x):
        x = self.main(x)
        return x


In [17]:
model=Regressor().to(device)
model.apply(weight_init)
# criterion = nn.BCELoss()
# criterion = nn.BCEWithLogitsLoss(reduction='sum',pos_weight=pos_weight).to(device)


Regressor(
  (main): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): ReLU(inplace=True)
    (8): Linear(in_features=64, out_features=64, bias=True)
    (9): ReLU(inplace=True)
    (10): Linear(in_features=64, out_features=32, bias=True)
    (11): ReLU(inplace=True)
    (12): Linear(in_features=32, out_features=32, bias=True)
    (13): ReLU(inplace=True)
    (14): Linear(in_features=32, out_features=16, bias=True)
    (15): ReLU(inplace=True)
    (16): Linear(in_features=16, out_features=4, bias=True)
  )
)

In [18]:
def train(**kwargs):
    torch.cuda.empty_cache()
    opt = Config()
    print('loading the model...')

    
    epoch_size = int(num_samples /opt.batch_size)  # number of training examples/batch size
#     optimizer = adamw.AdamW(model.parameters(), lr=opt.lr,weight_decay=1e-5)#weight_decay=1e-5  
    optimizer = torch.optim.Adam(model.parameters(), lr = opt.lr,weight_decay=5e-3)#,amsgrad=True
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=0.005 )
    
    #loading pretrained encoder
    en_state = torch.load(os.path.join('./checkpoint/vae_wbc/models', 'train_model.pth'),map_location='cuda:5')
    encoder.load_state_dict(en_state['enc_state_dict'])   

    
    try:
        state = torch.load(os.path.join(opt.model_path, 'model.pth'),map_location='cuda:5')
        model.load_state_dict(state['state_dict'])
        print("Loaded pre-trained model with success.")
        e_counter=state['epoch']
        best_acc = state['valid_acc_max']
        print('Previously Trained for {} epoches'.format(e_counter))
        print('Best val Accuracy till yet :',best_acc,'%')
        e_counter+=1
    except FileNotFoundError:
        print("Pre-trained weights not found. Training from scratch.")
        e_counter=0
        best_valid_loss = float('inf')
        prev_loss=float('inf')
        best_acc = 0.0

  
    for epoch in range(e_counter,opt.max_epochs):
        model.train()
        epoch_start_time = time.time()
        print()
        print('==================================================================')
        print('-------------Epoch: {}/{}------------'.format(epoch,opt.max_epochs))
        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss=0.0
            running_corrects = 0.0 
            running_auc=0.0
            
            p_c=0
            p_t=0
            p_acc=0
            
            r_acc=0
            r=0
            for idx,batch in enumerate(dataloaders[phase]):
                images,labels=batch
                images,labels=images.to(device),labels.to(device)
                
                l=labels.to("cpu").to(torch.int).numpy()
                pos_w = []
                for i in range(num_classes):
                    z=o=0
                    for j in range(opt.batch_size):
                        if l[j][i] == 0:
                            z += 1
                        elif l[j][i] == 1:
                            o += 1
                    if o == 0:
                        p_w = 1.0
                    else:
                        p_w = z/o
                    pos_w.append(p_w)
                
                pos_w= torch.FloatTensor(pos_w)
                criterion = nn.BCEWithLogitsLoss(reduction='sum',pos_weight=pos_w).to(device)
                
                if phase=='train':
                    optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    enc_out,mu,logvar = encoder(images)
                    outputs = model(mu)
                    loss = criterion(outputs, labels)
                    
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                

                preds=outputs.data > 0.5
#                 preds = torch.sigmoid(outputs).data > 0.5
#                 preds = outputs.data > 0.4
                preds = preds.to(torch.float32)
    
        
                auclist = []
                p=preds.to("cpu").to(torch.int).numpy()
                
                for i in range(opt.batch_size):
                    label=labels[i].to('cpu').to(torch.int).numpy()
                    pred= preds[i].to('cpu').to(torch.int).numpy()
                    p_t += len(label)*opt.batch_size
                    fpr,tpr,thresholds=roc_curve(label,pred,pos_label=1)
                    AUC = auc(fpr, tpr)
                    auclist.append(AUC)
                    for j in range(len(label)):
                        if label[j]==pred[j]:
                            p_c += 1
                    
                    p_acc += p_c/p_t
                    
                
                temp= np.array(auclist).mean()
                running_auc += temp
                
                running_loss += loss.item() 
                
                r_acc += accuracy_score(p, l)
            
#                 running_corrects += f1_score(labels.to("cpu").to(torch.int).numpy() ,
#                                              preds.to("cpu").to(torch.int).numpy() , 
#                                              average="samples")* images.size(0)
               
                
                if (idx+1)%500 == 0:  
                   # print(running_loss,r_acc,p_acc,running_auc)
                    print(f'{phase}_batch {idx+1}/{len(dataloaders[phase])} Loss: {(running_loss/((idx+1))):.5f}'
                              f' ACC: {(r_acc /(idx + 1))*100:.3f}%'
                              f' p_acc: {(p_acc /(idx + 1))*100:.3f}%'
                              f' AUC: {(running_auc /((idx + 1))):.3f}'
                         )
                    
            #print(running_loss,r_acc,p_acc,running_auc)   
            epoch_loss = running_loss / len(dataloaders[phase])
            epoch_acc = (r_acc / len(dataloaders[phase]))*100
            epoch_p_acc = (p_acc / len(dataloaders[phase]))*100                           
            epoch_auc = running_auc/ len(dataloaders[phase])
 
            print(f'{phase} Epoch Loss: {epoch_loss:.5f} ACC: {epoch_acc:.3f}% p_ACC: {epoch_p_acc:.3f}% AUC: {epoch_auc: .3f}')
            with open('./checkpoint/Integrated/train_logs.txt', 'a') as file:
                file.write('epoch: '+str(epoch)+',loss: '+str(epoch_loss)+',acc: '+str(epoch_acc)+',auc: '+str(epoch_auc) +'\n')
            print('.............')
            state = {
                    'epoch': epoch,
                    'valid_acc_max': epoch_acc,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict()
                    }

            if phase == 'val':
                if epoch_acc > best_acc:
                    
                    print('Val auc increased ({:.3f} --> {:.3f}). Saving model ...'.format(best_acc,epoch_acc))
                    torch.save(state, os.path.join(opt.model_path, 'model.pth'))
                    with open('./checkpoint/Integrated/val_logs.txt', 'a') as file:
                        file.write('epoch: ' + str(epoch) + ',loss: ' + str(epoch_loss) + ',acc: ' + str(epoch_acc)+',auc: '+str(epoch_auc) + '\n')
                        
                    best_acc = epoch_acc
                epoch_time = int(time.time() - epoch_start_time)
                print(f'-----------------Epoch cost time {epoch_time}s--------------------')
#                 scheduler.step()



In [ ]:
train()

loading the model...
Loaded pre-trained model with success.
Previously Trained for 18 epoches
Best val Accuracy till yet : 48.07923169267704 %

-------------Epoch: 19/500------------
train_batch 500/5000 Loss: 5.12556 ACC: 78.667% p_acc: 93.170% AUC: 0.928
train_batch 1000/5000 Loss: 5.29377 ACC: 77.967% p_acc: 93.032% AUC: 0.924
train_batch 1500/5000 Loss: 5.37821 ACC: 77.333% p_acc: 92.911% AUC: 0.921
train_batch 2000/5000 Loss: 5.54777 ACC: 76.642% p_acc: 92.795% AUC: 0.919
train_batch 2500/5000 Loss: 5.58712 ACC: 76.640% p_acc: 92.697% AUC: 0.919
train_batch 3000/5000 Loss: 5.62531 ACC: 76.617% p_acc: 92.635% AUC: 0.918
train_batch 3500/5000 Loss: 5.67663 ACC: 76.610% p_acc: 92.586% AUC: 0.918
train_batch 4000/5000 Loss: 5.78732 ACC: 76.383% p_acc: 92.541% AUC: 0.916
train_batch 4500/5000 Loss: 5.81404 ACC: 76.156% p_acc: 92.490% AUC: 0.915
train_batch 5000/5000 Loss: 5.92366 ACC: 75.727% p_acc: 92.438% AUC: 0.913
train Epoch Loss: 5.92366 ACC: 75.727% p_ACC: 92.438% AUC:  0.913
..

train_batch 500/5000 Loss: 4.01611 ACC: 81.433% p_acc: 94.148% AUC: 0.942
train_batch 1000/5000 Loss: 4.22957 ACC: 81.033% p_acc: 94.150% AUC: 0.940
train_batch 1500/5000 Loss: 4.37257 ACC: 80.967% p_acc: 94.116% AUC: 0.939
train_batch 2000/5000 Loss: 4.39567 ACC: 80.992% p_acc: 94.087% AUC: 0.939
train_batch 2500/5000 Loss: 4.55717 ACC: 80.447% p_acc: 94.045% AUC: 0.936
train_batch 3000/5000 Loss: 4.63143 ACC: 80.289% p_acc: 93.993% AUC: 0.935
train_batch 3500/5000 Loss: 4.61791 ACC: 80.186% p_acc: 93.949% AUC: 0.935
train_batch 4000/5000 Loss: 4.67046 ACC: 80.008% p_acc: 93.908% AUC: 0.934
train_batch 4500/5000 Loss: 4.72392 ACC: 79.907% p_acc: 93.869% AUC: 0.933
train_batch 5000/5000 Loss: 4.80335 ACC: 79.680% p_acc: 93.830% AUC: 0.932
train Epoch Loss: 4.80335 ACC: 79.680% p_ACC: 93.830% AUC:  0.932
.............
val_batch 500/833 Loss: 33.71514 ACC: 43.900% p_acc: 74.749% AUC: 0.705
val Epoch Loss: 34.14974 ACC: 44.858% p_ACC: 74.923% AUC:  0.711
.............
-----------------Epo

train_batch 2000/5000 Loss: 3.83252 ACC: 84.117% p_acc: 95.296% AUC: 0.948
train_batch 2500/5000 Loss: 3.94511 ACC: 83.547% p_acc: 95.213% AUC: 0.946
train_batch 3000/5000 Loss: 3.94990 ACC: 83.339% p_acc: 95.134% AUC: 0.946
train_batch 3500/5000 Loss: 4.02268 ACC: 83.033% p_acc: 95.069% AUC: 0.945
train_batch 4000/5000 Loss: 4.05571 ACC: 82.829% p_acc: 95.007% AUC: 0.944
train_batch 4500/5000 Loss: 4.08702 ACC: 82.607% p_acc: 94.954% AUC: 0.943
train_batch 5000/5000 Loss: 4.13092 ACC: 82.437% p_acc: 94.903% AUC: 0.942
train Epoch Loss: 4.13092 ACC: 82.437% p_ACC: 94.903% AUC:  0.942
.............
val_batch 500/833 Loss: 38.21800 ACC: 47.333% p_acc: 75.121% AUC: 0.719
val Epoch Loss: 38.47224 ACC: 46.899% p_ACC: 75.337% AUC:  0.718
.............
-----------------Epoch cost time 260s--------------------

-------------Epoch: 34/500------------
train_batch 500/5000 Loss: 3.17516 ACC: 85.533% p_acc: 95.500% AUC: 0.958
train_batch 1000/5000 Loss: 3.30448 ACC: 85.000% p_acc: 95.529% AUC: 0.9

train_batch 3500/5000 Loss: 3.63817 ACC: 83.805% p_acc: 95.315% AUC: 0.950
train_batch 4000/5000 Loss: 3.69063 ACC: 83.700% p_acc: 95.280% AUC: 0.949
train_batch 4500/5000 Loss: 3.70267 ACC: 83.696% p_acc: 95.249% AUC: 0.949
train_batch 5000/5000 Loss: 3.75723 ACC: 83.617% p_acc: 95.220% AUC: 0.948
train Epoch Loss: 3.75723 ACC: 83.617% p_ACC: 95.220% AUC:  0.948
.............
val_batch 500/833 Loss: 35.50766 ACC: 45.133% p_acc: 74.379% AUC: 0.698
val Epoch Loss: 35.33614 ACC: 44.738% p_ACC: 74.340% AUC:  0.697
.............
-----------------Epoch cost time 261s--------------------

-------------Epoch: 41/500------------
train_batch 500/5000 Loss: 3.37353 ACC: 85.867% p_acc: 95.625% AUC: 0.959
train_batch 1000/5000 Loss: 3.36283 ACC: 85.600% p_acc: 95.686% AUC: 0.957
train_batch 1500/5000 Loss: 3.44874 ACC: 85.178% p_acc: 95.649% AUC: 0.955
train_batch 2000/5000 Loss: 3.43909 ACC: 84.825% p_acc: 95.605% AUC: 0.954
train_batch 2500/5000 Loss: 3.50216 ACC: 84.827% p_acc: 95.559% AUC: 0.9

val Epoch Loss: 37.87540 ACC: 46.198% p_ACC: 75.969% AUC:  0.712
.............
-----------------Epoch cost time 261s--------------------

-------------Epoch: 48/500------------
train_batch 500/5000 Loss: 2.98442 ACC: 87.500% p_acc: 96.202% AUC: 0.960
train_batch 1000/5000 Loss: 3.00527 ACC: 86.967% p_acc: 96.139% AUC: 0.960
train_batch 1500/5000 Loss: 3.12284 ACC: 86.511% p_acc: 96.091% AUC: 0.958
train_batch 2000/5000 Loss: 3.15126 ACC: 86.217% p_acc: 96.034% AUC: 0.957
train_batch 2500/5000 Loss: 3.18860 ACC: 86.027% p_acc: 95.981% AUC: 0.956
train_batch 3000/5000 Loss: 3.30264 ACC: 85.767% p_acc: 95.931% AUC: 0.955
train_batch 3500/5000 Loss: 3.33876 ACC: 85.448% p_acc: 95.877% AUC: 0.954
train_batch 4000/5000 Loss: 3.39290 ACC: 85.329% p_acc: 95.829% AUC: 0.953
train_batch 4500/5000 Loss: 3.43107 ACC: 85.163% p_acc: 95.786% AUC: 0.953
train_batch 5000/5000 Loss: 3.45257 ACC: 85.070% p_acc: 95.748% AUC: 0.952
train Epoch Loss: 3.45257 ACC: 85.070% p_ACC: 95.748% AUC:  0.952
........

train_batch 2000/5000 Loss: 2.97561 ACC: 87.125% p_acc: 96.311% AUC: 0.960
train_batch 2500/5000 Loss: 3.00451 ACC: 86.900% p_acc: 96.246% AUC: 0.959
train_batch 3000/5000 Loss: 3.00723 ACC: 86.711% p_acc: 96.195% AUC: 0.959
train_batch 3500/5000 Loss: 3.04925 ACC: 86.433% p_acc: 96.151% AUC: 0.958
train_batch 4000/5000 Loss: 3.17215 ACC: 86.146% p_acc: 96.107% AUC: 0.957
train_batch 1000/5000 Loss: 2.79140 ACC: 88.333% p_acc: 96.857% AUC: 0.966
train_batch 1500/5000 Loss: 2.96097 ACC: 87.667% p_acc: 96.679% AUC: 0.962
train_batch 2000/5000 Loss: 3.05107 ACC: 87.242% p_acc: 96.548% AUC: 0.960
train_batch 2500/5000 Loss: 3.10126 ACC: 87.047% p_acc: 96.448% AUC: 0.959
train_batch 3000/5000 Loss: 3.17636 ACC: 86.578% p_acc: 96.356% AUC: 0.957
train_batch 3500/5000 Loss: 3.24689 ACC: 86.390% p_acc: 96.278% AUC: 0.956
train_batch 4000/5000 Loss: 3.27743 ACC: 86.221% p_acc: 96.210% AUC: 0.956
train_batch 4500/5000 Loss: 3.29941 ACC: 85.944% p_acc: 96.149% AUC: 0.955
train_batch 5000/5000 Los

train_batch 500/5000 Loss: 2.87020 ACC: 88.300% p_acc: 96.339% AUC: 0.964
train_batch 1000/5000 Loss: 2.88991 ACC: 88.383% p_acc: 96.413% AUC: 0.963
train_batch 1500/5000 Loss: 2.92842 ACC: 88.044% p_acc: 96.384% AUC: 0.962
train_batch 2000/5000 Loss: 2.98716 ACC: 87.650% p_acc: 96.354% AUC: 0.961
train_batch 2500/5000 Loss: 3.00011 ACC: 87.707% p_acc: 96.319% AUC: 0.961
train_batch 3000/5000 Loss: 3.05181 ACC: 87.417% p_acc: 96.285% AUC: 0.960
train_batch 3500/5000 Loss: 3.06555 ACC: 87.252% p_acc: 96.252% AUC: 0.959
train_batch 4000/5000 Loss: 3.12390 ACC: 87.088% p_acc: 96.217% AUC: 0.959
train_batch 4500/5000 Loss: 3.17883 ACC: 86.859% p_acc: 96.183% AUC: 0.958
train_batch 5000/5000 Loss: 3.20426 ACC: 86.760% p_acc: 96.151% AUC: 0.957
train Epoch Loss: 3.20426 ACC: 86.760% p_ACC: 96.151% AUC:  0.957
.............
val_batch 500/833 Loss: 40.09130 ACC: 43.633% p_acc: 73.421% AUC: 0.695
val Epoch Loss: 39.99520 ACC: 43.457% p_ACC: 73.637% AUC:  0.692
.............
-----------------Epo

train_batch 2000/5000 Loss: 2.59610 ACC: 89.200% p_acc: 97.008% AUC: 0.967
train_batch 2500/5000 Loss: 2.66836 ACC: 88.873% p_acc: 96.940% AUC: 0.966
train_batch 3000/5000 Loss: 2.75654 ACC: 88.611% p_acc: 96.878% AUC: 0.965
train_batch 3500/5000 Loss: 2.80369 ACC: 88.371% p_acc: 96.822% AUC: 0.964
train_batch 4000/5000 Loss: 2.83404 ACC: 88.229% p_acc: 96.772% AUC: 0.963
train_batch 4500/5000 Loss: 2.90753 ACC: 87.956% p_acc: 96.725% AUC: 0.962
train_batch 5000/5000 Loss: 2.97092 ACC: 87.713% p_acc: 96.678% AUC: 0.961
train Epoch Loss: 2.97092 ACC: 87.713% p_ACC: 96.678% AUC:  0.961
.............
val_batch 500/833 Loss: 40.37115 ACC: 45.900% p_acc: 76.048% AUC: 0.711
val Epoch Loss: 40.61529 ACC: 44.738% p_ACC: 75.712% AUC:  0.706
.............
-----------------Epoch cost time 261s--------------------

-------------Epoch: 71/500------------
train_batch 500/5000 Loss: 2.33236 ACC: 90.000% p_acc: 96.949% AUC: 0.967
train_batch 1000/5000 Loss: 2.48547 ACC: 89.600% p_acc: 96.883% AUC: 0.9

train_batch 3500/5000 Loss: 2.80613 ACC: 88.652% p_acc: 96.921% AUC: 0.964
train_batch 4000/5000 Loss: 2.82989 ACC: 88.621% p_acc: 96.865% AUC: 0.963
train_batch 4500/5000 Loss: 2.81911 ACC: 88.530% p_acc: 96.817% AUC: 0.963
train_batch 5000/5000 Loss: 2.86018 ACC: 88.370% p_acc: 96.776% AUC: 0.962
train Epoch Loss: 2.86018 ACC: 88.370% p_ACC: 96.776% AUC:  0.962
.............
val_batch 500/833 Loss: 41.06756 ACC: 45.433% p_acc: 75.230% AUC: 0.705
val Epoch Loss: 40.80408 ACC: 45.438% p_ACC: 75.293% AUC:  0.711
.............
-----------------Epoch cost time 261s--------------------

-------------Epoch: 78/500------------
train_batch 500/5000 Loss: 2.16917 ACC: 90.667% p_acc: 97.142% AUC: 0.971
train_batch 1000/5000 Loss: 2.43898 ACC: 89.567% p_acc: 97.126% AUC: 0.967
train_batch 1500/5000 Loss: 2.51450 ACC: 89.189% p_acc: 97.000% AUC: 0.967
train_batch 2000/5000 Loss: 2.59610 ACC: 88.958% p_acc: 96.921% AUC: 0.966
train_batch 2500/5000 Loss: 2.59220 ACC: 88.993% p_acc: 96.869% AUC: 0.9

val_batch 500/833 Loss: 38.16749 ACC: 42.867% p_acc: 74.676% AUC: 0.699
val Epoch Loss: 39.52135 ACC: 42.597% p_ACC: 74.476% AUC:  0.699
.............
-----------------Epoch cost time 157s--------------------

-------------Epoch: 85/500------------
train_batch 500/5000 Loss: 2.32550 ACC: 89.833% p_acc: 96.975% AUC: 0.969
train_batch 1000/5000 Loss: 2.17140 ACC: 90.800% p_acc: 97.056% AUC: 0.971
